Import ONI data

In [1]:
import pandas as pd
import os

oni_data = pd.read_csv("lawas/oni.txt", sep="\s+")
oni_data

,YR,MON,TOTAL,ClimAdjust,ANOM
0,1950,1,24.56,26.18,-1.62
1,1950,2,25.07,26.39,-1.32
2,1950,3,25.88,26.95,-1.07
3,1950,4,26.29,27.39,-1.11
4,1950,5,26.19,27.56,-1.37
...,...,...,...,...,...
893,2024,6,27.91,27.73,0.18
894,2024,7,27.34,27.29,0.05
895,2024,8,26.74,26.86,-0.11
896,2024,9,26.47,26.72,-0.25


Import DID data

In [2]:
import pandas as pd
import os
import re

precipitation_data = pd.DataFrame()
for file in os.listdir("lawas"):
    if re.search(r"rainfall-data\d+\.csv", file):
        df = pd.read_csv(f"lawas/{file}")
        precipitation_data = pd.concat([precipitation_data, df], ignore_index=True)

In [3]:
precipitation_data["DateTime"] = pd.to_datetime(precipitation_data["DateTime"],format="%Y%m%d %H%M")
precipitation_data = precipitation_data[["DateTime", "Rainfall (mm)"]]
precipitation_data.rename(columns={"Rainfall (mm)": "Rainfall"}, inplace=True)
precipitation_data

,DateTime,Rainfall
0,1998-07-13 12:00:00,1.0
1,1998-07-13 13:00:00,2.0
2,1998-07-13 14:00:00,0.0
3,1998-07-13 15:00:00,0.0
4,1998-07-13 16:00:00,0.0
...,...,...
200096,2022-04-07 20:00:00,1.0
200097,2022-04-07 21:00:00,0.0
200098,2022-04-07 22:00:00,0.0
200099,2022-04-07 23:00:00,0.0


Merge DID and ONI data

In [4]:
precipitation_data = pd.merge(
    precipitation_data,
    oni_data[["YR", "MON", "TOTAL", "ClimAdjust", "ANOM"]],
    left_on=[precipitation_data["DateTime"].dt.year, precipitation_data["DateTime"].dt.month],
    right_on= ["YR", "MON"]
).drop(["YR", "MON"], axis=1)
precipitation_data.set_index("DateTime",inplace=True)
precipitation_data


,Rainfall,TOTAL,ClimAdjust,ANOM
DateTime,,,,
1998-07-13 12:00:00,1.0,26.32,27.18,-0.86
1998-07-13 13:00:00,2.0,26.32,27.18,-0.86
1998-07-13 14:00:00,0.0,26.32,27.18,-0.86
1998-07-13 15:00:00,0.0,26.32,27.18,-0.86
1998-07-13 16:00:00,0.0,26.32,27.18,-0.86
...,...,...,...,...
2022-04-07 20:00:00,1.0,26.72,27.83,-1.11
2022-04-07 21:00:00,0.0,26.72,27.83,-1.11
2022-04-07 22:00:00,0.0,26.72,27.83,-1.11


Import Wunderground data

In [5]:
feature_data = pd.read_csv("lawas/rainfall-feature-wunderground.csv")
feature_data.rename(columns={"Time":"DateTime"},inplace=True)
feature_data["DateTime"] = pd.to_datetime(feature_data["DateTime"], format="%Y-%m-%d %H:%M:%S%z")
feature_data["DateTime"] = feature_data["DateTime"].dt.tz_localize(None)
feature_data.set_index("DateTime",inplace=True)
feature_data.drop(columns={"Wind Gust", "Precip."}, inplace=True)
feature_data.fillna({"Wind Speed":0}, inplace=True)
feature_data.ffill(inplace=True)
feature_data.rename(columns={"Dew Point": "DewPoint", "Wind Speed": "WindSpeed"}, inplace=True)
feature_data

,Temperature,DewPoint,Humidity,Wind,WindSpeed,Pressure,Condition
DateTime,,,,,,,
2002-08-01 00:00:00,79.0,75.0,89.0,E,3.0,29.69,Mostly Cloudy
2002-08-01 01:00:00,79.0,73.0,83.0,E,2.0,29.69,Mostly Cloudy
2002-08-01 02:00:00,77.0,75.0,94.0,W,2.0,29.66,Mostly Cloudy
2002-08-01 03:00:00,77.0,73.0,89.0,CALM,0.0,29.66,Mostly Cloudy
2002-08-01 04:00:00,79.0,73.0,83.0,NE,1.0,29.63,Mostly Cloudy
...,...,...,...,...,...,...,...
2022-04-07 19:00:00,79.0,75.0,89.0,W,3.0,29.66,Fair
2022-04-07 20:00:00,79.0,77.0,94.0,VAR,1.0,29.69,Fair
2022-04-07 21:00:00,79.0,77.0,94.0,VAR,2.0,29.72,Fair


Combine all data

In [6]:
lawas_rainfall = precipitation_data.join(feature_data)
lawas_rainfall.dropna(inplace=True)
lawas_rainfall

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,Wind,WindSpeed,Pressure,Condition
DateTime,,,,,,,,,,,
2002-08-01 00:00:00,0.0,27.72,26.91,0.81,79.0,75.0,89.0,E,3.0,29.69,Mostly Cloudy
2002-08-01 01:00:00,0.0,27.72,26.91,0.81,79.0,73.0,83.0,E,2.0,29.69,Mostly Cloudy
2002-08-01 02:00:00,0.0,27.72,26.91,0.81,77.0,75.0,94.0,W,2.0,29.66,Mostly Cloudy
2002-08-01 03:00:00,0.0,27.72,26.91,0.81,77.0,73.0,89.0,CALM,0.0,29.66,Mostly Cloudy
2002-08-01 04:00:00,0.0,27.72,26.91,0.81,79.0,73.0,83.0,NE,1.0,29.63,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 19:00:00,0.5,26.72,27.83,-1.11,79.0,75.0,89.0,W,3.0,29.66,Fair
2022-04-07 20:00:00,1.0,26.72,27.83,-1.11,79.0,77.0,94.0,VAR,1.0,29.69,Fair
2022-04-07 21:00:00,0.0,26.72,27.83,-1.11,79.0,77.0,94.0,VAR,2.0,29.72,Fair


Check for duplicate data

In [7]:
duplicates = lawas_rainfall.index[lawas_rainfall.index.duplicated()]
if not duplicates.empty:
    print("Duplicate timestamps found:", duplicates)



Duplicate timestamps found: DatetimeIndex(['2006-08-02 17:00:00', '2006-08-02 18:00:00',
               '2006-08-02 19:00:00', '2017-01-14 01:00:00',
               '2020-12-03 00:00:00', '2020-12-03 01:00:00',
               '2020-12-03 02:00:00', '2020-12-03 03:00:00',
               '2020-12-03 04:00:00', '2020-12-03 05:00:00',
               '2020-12-03 06:00:00', '2020-12-03 07:00:00',
               '2020-12-03 08:00:00', '2020-12-03 09:00:00',
               '2020-12-03 10:00:00', '2020-12-03 11:00:00',
               '2020-12-03 12:00:00', '2020-12-03 13:00:00',
               '2020-12-03 14:00:00', '2020-12-03 15:00:00',
               '2020-12-03 16:00:00'],
              dtype='datetime64[ns]', name='DateTime', freq=None)


Find the data losses


In [8]:
datetime_range = pd.date_range(start=lawas_rainfall.index.min(), end=lawas_rainfall.index.max(), freq='1h')

missing_date = datetime_range.difference(lawas_rainfall.index)
missing_date

DatetimeIndex(['2002-08-21 09:00:00', '2002-08-21 10:00:00',
               '2002-08-21 11:00:00', '2002-09-10 08:00:00',
               '2002-09-10 12:00:00', '2002-09-10 13:00:00',
               '2002-09-10 14:00:00', '2002-09-10 15:00:00',
               '2002-09-10 16:00:00', '2002-09-18 14:00:00',
               ...
               '2021-12-31 19:00:00', '2021-12-31 20:00:00',
               '2021-12-31 21:00:00', '2021-12-31 22:00:00',
               '2021-12-31 23:00:00', '2022-01-15 13:00:00',
               '2022-01-15 14:00:00', '2022-02-01 08:00:00',
               '2022-02-01 09:00:00', '2022-02-01 10:00:00'],
              dtype='datetime64[ns]', length=10173, freq=None)

In [9]:
start_date = "2002-08-01"
end_date = "2021-01-31"
# There is data losses from Feb 2021 - Dec 2021, as there is major loss. The data will be use from Aug 2002 until Jan 2021

missing_date[(missing_date >= start_date) & (missing_date <= end_date)]

DatetimeIndex(['2002-08-21 09:00:00', '2002-08-21 10:00:00',
               '2002-08-21 11:00:00', '2002-09-10 08:00:00',
               '2002-09-10 12:00:00', '2002-09-10 13:00:00',
               '2002-09-10 14:00:00', '2002-09-10 15:00:00',
               '2002-09-10 16:00:00', '2002-09-18 14:00:00',
               ...
               '2020-11-08 23:00:00', '2020-11-10 01:00:00',
               '2020-11-16 01:00:00', '2021-01-12 00:00:00',
               '2021-01-12 01:00:00', '2021-01-12 02:00:00',
               '2021-01-12 03:00:00', '2021-01-12 04:00:00',
               '2021-01-12 05:00:00', '2021-01-28 20:00:00'],
              dtype='datetime64[ns]', length=2213, freq=None)

In [10]:
numeric_cols = lawas_rainfall.select_dtypes(include=['number']).columns
non_numeric_cols = lawas_rainfall.select_dtypes(exclude=['number']).columns

# Handle duplicate index by grouping
# For numeric columns, take the mean
# For non-numeric columns, take the first occurrence (or other appropriate method)
lawas_rainfall = (
    lawas_rainfall.groupby(lawas_rainfall.index)
    .agg({**{col: 'mean' for col in numeric_cols},
          **{col: 'first' for col in non_numeric_cols}})
)
lawas_rainfall

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,WindSpeed,Pressure,Wind,Condition
DateTime,,,,,,,,,,,
2002-08-01 00:00:00,0.0,27.72,26.91,0.81,79.0,75.0,89.0,3.0,29.69,E,Mostly Cloudy
2002-08-01 01:00:00,0.0,27.72,26.91,0.81,79.0,73.0,83.0,2.0,29.69,E,Mostly Cloudy
2002-08-01 02:00:00,0.0,27.72,26.91,0.81,77.0,75.0,94.0,2.0,29.66,W,Mostly Cloudy
2002-08-01 03:00:00,0.0,27.72,26.91,0.81,77.0,73.0,89.0,0.0,29.66,CALM,Mostly Cloudy
2002-08-01 04:00:00,0.0,27.72,26.91,0.81,79.0,73.0,83.0,1.0,29.63,NE,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-07 19:00:00,0.5,26.72,27.83,-1.11,79.0,75.0,89.0,3.0,29.66,W,Fair
2022-04-07 20:00:00,1.0,26.72,27.83,-1.11,79.0,77.0,94.0,1.0,29.69,VAR,Fair
2022-04-07 21:00:00,0.0,26.72,27.83,-1.11,79.0,77.0,94.0,2.0,29.72,VAR,Fair


In [11]:
lawas_rainfall = lawas_rainfall.reindex(
                    pd.date_range(start=start_date, end=end_date,
                    freq='1h'),
                    method='ffill'
                    )
lawas_rainfall.index.name = "DateTime"
lawas_rainfall

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,WindSpeed,Pressure,Wind,Condition
DateTime,,,,,,,,,,,
2002-08-01 00:00:00,0.0,27.72,26.91,0.81,79.0,75.0,89.0,3.0,29.69,E,Mostly Cloudy
2002-08-01 01:00:00,0.0,27.72,26.91,0.81,79.0,73.0,83.0,2.0,29.69,E,Mostly Cloudy
2002-08-01 02:00:00,0.0,27.72,26.91,0.81,77.0,75.0,94.0,2.0,29.66,W,Mostly Cloudy
2002-08-01 03:00:00,0.0,27.72,26.91,0.81,77.0,73.0,89.0,0.0,29.66,CALM,Mostly Cloudy
2002-08-01 04:00:00,0.0,27.72,26.91,0.81,79.0,73.0,83.0,1.0,29.63,NE,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-30 20:00:00,0.5,25.56,26.55,-0.99,79.0,73.0,83.0,13.0,29.72,NW,Fair
2021-01-30 21:00:00,0.0,25.56,26.55,-0.99,79.0,75.0,89.0,12.0,29.72,NW,Light Rain
2021-01-30 22:00:00,0.0,25.56,26.55,-0.99,81.0,75.0,84.0,13.0,29.72,NW,Fair


In [12]:
# Check on start and end data
print(f"Head Data:\n{lawas_rainfall.head()}\n")
print(f"Tail Data:\n{lawas_rainfall.tail()}")

Head Data:
                     Rainfall  TOTAL  ClimAdjust  ANOM  Temperature  DewPoint  \
DateTime                                                                        
2002-08-01 00:00:00       0.0  27.72       26.91  0.81         79.0      75.0   
2002-08-01 01:00:00       0.0  27.72       26.91  0.81         79.0      73.0   
2002-08-01 02:00:00       0.0  27.72       26.91  0.81         77.0      75.0   
2002-08-01 03:00:00       0.0  27.72       26.91  0.81         77.0      73.0   
2002-08-01 04:00:00       0.0  27.72       26.91  0.81         79.0      73.0   

                     Humidity  WindSpeed  Pressure  Wind      Condition  
DateTime                                                                 
2002-08-01 00:00:00      89.0        3.0     29.69     E  Mostly Cloudy  
2002-08-01 01:00:00      83.0        2.0     29.69     E  Mostly Cloudy  
2002-08-01 02:00:00      94.0        2.0     29.66     W  Mostly Cloudy  
2002-08-01 03:00:00      89.0        0.0     29.66 

In [13]:
lawas_rainfall.columns

Index(['Rainfall', 'TOTAL', 'ClimAdjust', 'ANOM', 'Temperature', 'DewPoint',
       'Humidity', 'WindSpeed', 'Pressure', 'Wind', 'Condition'],
      dtype='object')

Downsample data - daily, weekly

In [14]:
lawas_rainfall_daily = lawas_rainfall.resample('D').agg({
    'Rainfall': 'sum',
    'TOTAL': 'mean',
    'ClimAdjust': 'mean',
    'ANOM': 'mean',
    'Temperature': 'mean',
    'DewPoint': 'mean',
    'Humidity': 'mean',
    'WindSpeed': 'mean',
    'Pressure': 'mean',
    'Wind': lambda x: x.mode()[0],
    'Condition': lambda x: x.mode()[0]
})

lawas_rainfall_weekly = lawas_rainfall.resample('W').agg({
    'Rainfall': 'sum',
    'TOTAL': 'mean',
    'ClimAdjust': 'mean',
    'ANOM': 'mean',
    'Temperature': 'mean',
    'DewPoint': 'mean',
    'Humidity': 'mean',
    'WindSpeed': 'mean',
    'Pressure': 'mean',
    'Wind': lambda x: x.mode()[0],
    'Condition': lambda x: x.mode()[0]
})

In [15]:
lawas_rainfall_daily

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,WindSpeed,Pressure,Wind,Condition
DateTime,,,,,,,,,,,
2002-08-01,0.5,27.72,26.91,0.81,83.500000,73.500000,73.208333,3.625000,29.63250,E,Mostly Cloudy
2002-08-02,46.0,27.72,26.91,0.81,82.666667,75.416667,80.375000,6.666667,29.65250,NE,Mostly Cloudy
2002-08-03,23.0,27.72,26.91,0.81,81.958333,74.250000,78.833333,6.041667,29.67875,SSE,Mostly Cloudy
2002-08-04,9.5,27.72,26.91,0.81,82.791667,73.000000,73.458333,5.750000,29.67625,WNW,Mostly Cloudy
2002-08-05,1.5,27.72,26.91,0.81,81.708333,74.125000,79.500000,6.083333,29.69750,NW,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-27,0.0,25.56,26.55,-0.99,81.333333,74.833333,81.875000,6.083333,29.66250,WNW,Fair
2021-01-28,61.5,25.56,26.55,-0.99,81.958333,76.250000,83.375000,7.416667,29.68750,WNW,Fair
2021-01-29,4.0,25.56,26.55,-0.99,82.541667,74.041667,76.875000,6.958333,29.67500,NW,Fair


In [16]:
lawas_rainfall_weekly

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,WindSpeed,Pressure,Wind,Condition
DateTime,,,,,,,,,,,
2002-08-04,79.0,27.720000,26.910000,0.810000,82.729167,74.041667,76.468750,5.520833,29.660000,SSE,Mostly Cloudy
2002-08-11,71.0,27.720000,26.910000,0.810000,82.898810,73.226190,73.726190,6.255952,29.714821,W,Mostly Cloudy
2002-08-18,30.5,27.720000,26.910000,0.810000,82.416667,71.273810,70.779762,9.464286,29.648929,SW,Mostly Cloudy
2002-08-25,12.0,27.720000,26.910000,0.810000,82.910714,74.166667,76.488095,4.940476,29.715536,E,Mostly Cloudy
2002-09-01,66.5,27.732857,26.894286,0.838571,81.261905,74.047619,80.154762,5.017857,29.718214,S,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-03,256.5,25.497143,26.578571,-1.081429,80.738095,75.726190,85.636905,2.297619,29.659464,VAR,Fair
2021-01-10,65.5,25.560000,26.550000,-0.990000,81.196429,75.726190,84.285714,3.482143,29.639464,VAR,Fair
2021-01-17,143.5,25.560000,26.550000,-0.990000,78.541667,74.458333,88.113095,4.541667,29.671250,VAR,Fair


In [17]:
lawas_rainfall

,Rainfall,TOTAL,ClimAdjust,ANOM,Temperature,DewPoint,Humidity,WindSpeed,Pressure,Wind,Condition
DateTime,,,,,,,,,,,
2002-08-01 00:00:00,0.0,27.72,26.91,0.81,79.0,75.0,89.0,3.0,29.69,E,Mostly Cloudy
2002-08-01 01:00:00,0.0,27.72,26.91,0.81,79.0,73.0,83.0,2.0,29.69,E,Mostly Cloudy
2002-08-01 02:00:00,0.0,27.72,26.91,0.81,77.0,75.0,94.0,2.0,29.66,W,Mostly Cloudy
2002-08-01 03:00:00,0.0,27.72,26.91,0.81,77.0,73.0,89.0,0.0,29.66,CALM,Mostly Cloudy
2002-08-01 04:00:00,0.0,27.72,26.91,0.81,79.0,73.0,83.0,1.0,29.63,NE,Mostly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-30 20:00:00,0.5,25.56,26.55,-0.99,79.0,73.0,83.0,13.0,29.72,NW,Fair
2021-01-30 21:00:00,0.0,25.56,26.55,-0.99,79.0,75.0,89.0,12.0,29.72,NW,Light Rain
2021-01-30 22:00:00,0.0,25.56,26.55,-0.99,81.0,75.0,84.0,13.0,29.72,NW,Fair


Export as .csv file

In [18]:
lawas_rainfall.to_csv("lawas/lawas-rainfall.csv",index=True)
lawas_rainfall_daily.to_csv("lawas/lawas-rainfall-daily.csv",index=True)
lawas_rainfall_weekly.to_csv("lawas/lawas-rainfall-weekly.csv",index=True)